## 2. Elevation Data using SRTM

In [1]:
# Download stored variables from previous notebook

%store -r habitat_suitability_data_dir usfs_grasslands_path 
%store -r comanche_grassland_gdf pawnee_grassland_gdf usfs_grasslands_gdf

In [ ]:
# Prepare for download Part 1 of 1
## Import packages that will help with...

# Reproducible file paths
import os # Reproducible file paths
import pathlib # Find the home folder
from glob import glob  # returns list of paths
import zipfile # Work with zip files

# Find files by pattern
import matplotlib.pyplot as plt # Overlay pandas and xarry plots,Overlay raster and vector data
import rioxarray as rxr # Work with geospatial raster data


# Work with tabular, vector, and raster data
import cartopy.crs as ccrs # CRSs (Coordinate Reference Systems)
import geopandas as gpd # work with vector data
import hvplot.pandas # Interactive tabular and vector data
import hvplot.xarray # Interactive raster
from math import floor, ceil # working with bounds, floor rounds down ciel rounds up
import pandas as pd # Group and aggregate
from rioxarray.merge import merge_arrays # Merge rasters
import xarray as xr # Adjust images
import xrspatial # calculate slope

# Access NASA data
import earthaccess # Access NASA data from the cloud

In [ ]:
# Prep for downloading SRTM 

# Create data dir 
elevation_dir= os.path.join(habitat_suitability_data_dir, 'srtm')
os.makedirs(elevation_dir, exist_ok=True)

# call the variable to check location
elevation_dir

In [4]:
# bounds_gdfs
#comanche_grassland_gdf = gpd.read_file('path_to_shapefile_area1.shp'),
#pawnee_grassland_gdf = gpd.read_file('path_to_shapefile_area2.shp')
chosen_grasslands_bounds_gdfs = [comanche_grassland_gdf, pawnee_grassland_gdf]

In [ ]:
# Download Raster data through earthaccess Part 1 of 1
# Login and search earthaccess, download results

# login to earthaccess
earthaccess.login(strategy="interactive", persist=True)

# bounds_gdfs
chosen_grasslands_bounds_gdfs = [comanche_grassland_gdf, pawnee_grassland_gdf]

# Iterate through the list of bounding GeoDataFrames (areas of interest)
for bounds_gdf in chosen_grasslands_bounds_gdfs:

    # Only download once - conditional
    #if not glob (os.path.join(elevation_dir, '*hgt.zip')):
    # *when I used this my code wouldn't work*
   
    bounds = tuple(bounds_gdf.total_bounds)

    # Search earthaccess
    elevation_results = earthaccess.search_data(
        short_name = "SRTMGL1",
        bounding_box = bounds
    )
    elevation_results

    # Download earthaccess results
    srtm_files = earthaccess.download(elevation_results, elevation_dir)

    # Return a list of file paths that match the pattern
    srtm_files= glob (os.path.join(
    elevation_dir, 
    '*hgt.zip')
    )

# Call srtm_file to see it 
srtm_files

In [ ]:
# Create list of files for each study area
comanche_srtm_files = [
    srtm_files[0], 
    srtm_files[1], 
    srtm_files[2], 
    srtm_files[3], 
    srtm_files[4], 
    srtm_files[5]
    ]
pawnee_srtm_files = [
    srtm_files[6], 
    srtm_files[7], 
    srtm_files[8], 
    srtm_files[9]
    ]

# Call each list to make sure it's right
comanche_srtm_files , pawnee_srtm_files

In [ ]:
# Create function with description to process srtm raster images
# Part 1 of 1
def process_image_list(url_list, chosen_buffer, bounds_gdfs):
    """
    Load, crop, and scale a raster image 

    Parameters
    ----------
    url: file-like or path-like
      File accessor downloaded or obtained 
    chosen_buffer: float number
      Amount of degrees to extend past the bounds of the bounds_gdf 
    bounds_gdf: gpd.GeoDataFrame
      Area of interest to crop to

    Returns
    -------
    cropped_da: rxr.DataArray
      Processed raster
    """   

    # List to store paths of saved raster files
    #saved_files = []  

     # Iterate through the list of bounding GeoDataFrames (areas of interest)
    for bounds_gdf in bounds_gdfs:
        
      # List to store cropped DataArrays for the current site 
      da_list= []
      
      buffer= chosen_buffer

      for url in url_list:

          # Connect to the raster image
          da = rxr.open_rasterio(
            url, 
            mask_and_scale=True
            ).squeeze()
          
           # Get the study bounds
          bounds_min_lon, bounds_min_lat, bounds_max_lon, bounds_max_lat = (
            bounds_gdf
            .to_crs(da.rio.crs)
            .total_bounds 
            )

          # Crop the raster image to the bounds of the study area
          cropped_da = (
            da.rio.clip_box(bounds_min_lon-buffer, bounds_min_lat-buffer, bounds_max_lon+buffer, bounds_max_lat+buffer)
            )
          
          # Append the cropped DataArray to the list
          da_list.append(cropped_da)

          # Merge the cropped DataArrays for this site
          merged_da = (
            merge_arrays(da_list)
            )
      
      # Save the merged raster to the output directory
      #output_file = os.path.join(output_dir, os.path.basename(raster_path))
      #harmonized_raster.rio.to_raster(output_file)

      # Print confirmation message 
      #print(f" raster saved to: {output_file}")
      #harmonized_files.append(output_file)

      # Append the file path to the list of saved files
      #saved_files.append(output_file)
      
      return merged_da


srtm_comanche_result = process_image_list(comanche_srtm_files, .025, chosen_grasslands_bounds_gdfs)

srtm_comanche_result

In [ ]:
srtm_pawnee_result = process_image_list(pawnee_srtm_files, .025, chosen_grasslands_bounds_gdfs)

In [ ]:
# Process data using the process_image_list function

# Test the function on 1 study area
srtm_comanche_result.plot(
    cbar_kwargs={"label": "Elevation (meters)"},
    robust=True,
    cmap='terrain',
)
# Overlay the boundary of the same study area
comanche_grassland_gdf.boundary.plot(ax=plt.gca(),
    color='black').set(
        title='Comanche Grassland - Elevation ',
        xlabel='Longitude', 
        ylabel='Latitude',
        xticks=[],
        yticks=[] 
    )
plt.show()


In [ ]:
# Process data using the process_image_list function

# Test the function on 1 study area
srtm_pawnee_result.plot(
    cbar_kwargs={"label": "Elevation (meters)"},
    robust=True,
    cmap='terrain',
)
# Overlay the boundary of the same study area
pawnee_grassland_gdf.boundary.plot(ax=plt.gca(),
    color='black').set(
        title='Pawnee Grassland - Elevation ',
        xlabel='Longitude', 
        ylabel='Latitude',
        xticks=[],
        yticks=[] 
    )
plt.show()

In [ ]:

# Create a list to save the srtm results to
srtm_results = [
    srtm_comanche_result,
    srtm_pawnee_result
]

# Iterate through a list of sites 
for srtm_result in srtm_results:

    # Reproject into epsg utm zone
    utm13_epsg = 32613
    srtm_proj_da = srtm_result.rio.reproject(utm13_epsg)

    # Calculate slope
    slope_da = xrspatial.slope(srtm_proj_da)

    print(slope_da)



In [ ]:
# Test to make sure the slope function worked by plotting
slope_da.plot(
    cbar_kwargs={"label": "Slope (degrees)"},
    cmap='terrain',
)
comanche_grassland_gdf.to_crs(utm13_epsg).boundary.plot(
    ax=plt.gca(),
    color='white').set(
        title='Comanche Grassland - Caluclated Slope ',
        xlabel='Longitude', 
        ylabel='Latitude',
        xticks=[],
        yticks=[] 
    )
plt.show()

In [ ]:
# Test to make sure the slope function worked by plotting
slope_da.plot(
    cbar_kwargs={"label": "Slope (degrees)"},
    cmap='terrain',
)
pawnee_grassland_gdf.to_crs(utm13_epsg).boundary.plot(
    ax=plt.gca(),
    color='white').set(
        title='Pawnee Grassland - Caluclated Slope ',
        xlabel='Longitude', 
        ylabel='Latitude',
        xticks=[],
        yticks=[] 
    )
plt.show()